## Let’s recap

In [the fist lab](1-WKSHP-Terraform101-CreatingResources.ipynb), we covered HPE GreenLake with its Private Cloud Service and showed how to get started with Terraform and the Terraform provider for HPE GreenLake. In this lab, we will start with the same VM configuration created in Lab 1 as shown below. From there, we will show you how to tap into more advanced functionality that’s provided by Terraform and the HPE GreenLake provider:

![GreenLake VM created using Terraform](./Pictures/vmcreated.png "GreenLake VM created using Terraform")

First, let's reload our environment variables:

In [ ]:
# Check the content of student.src
cat student.src
# and source it
source ./student.src

## Infrastructure lifecycle management

Once your infrastructure is created, it will need to evolve over time in order to cope with any change of workload or to perform maintenance. Let’s look at a few scenarios.

### Use case 1: Setting up tags and labels

As organizations scale their cloud environments, they often need to define methodologies for organizing resources. For this, they can leverage tags and labels. Tags consist of key/value pairs that make it easier to search for, or filter, your cloud resources based on categories relevant to the organization. Another option is to attach labels, which are simple values, to your VMs in order to keep track of what it’s used for or who it belongs to.

So, let's try adding metadata to the VM using tags and labels. According to the [documentation](https://github.com/HPE/terraform-provider-hpegl/blob/main/docs/resources/vmaas_instance.md), you can add labels by adding the following snippets to our configuration file within the VM resource block: 

```markdown
# Using labels
labels = ["hackshack", "hpedev"]
```

And you can add tags by inserting the following other code snippet in your configuration file:

```markdown
# Using tags
tags = {
        team  = "HPE Developer"
        support = "gold"
  }
```

Save TF file and apply those changes with **terraform apply**. Run a **terraform validate** to make sure there is no problem with your file.

In [ ]:
terraform validate
terraform apply -input=false -auto-approve

Look again in your HPE GreenLake console at the VM details. You should see the labels and the tags as illustrated in the capture below:

![tags and labels applied to VM](./Pictures/greenlaketagandlabels.png "tags and labels applied to VM")

### Use case 2: Get me more disk space

Another typical use case would be to add another disk to a VM, say a data volume for application usage. The syntax for this is the same as what you used to create the VM with its *root_vol*, already visible in the Storage details of the VM:

![VM was created with one disk: root_vol](./Pictures/vmwithonedisk.png "VM was created with one disk: root_vol")

Go ahead and add the following code snippet to your configuration file, right underneath the first volume description:

```markdown
  # Add another volume 
  volume {
         name         = "data_vol"
         size         = 25
         datastore_id = "auto"
     }
```

Save the file, validate and apply again:

In [ ]:
terraform validate
terraform apply -input=false -auto-approve

Check again in your HPE GreenLake console, and you should now see two disks attached to your VM, as shown below:

![VM with two disks](./Pictures/vmwithtwodisks.png "VM with two disks")

### Use case 3: Please snap this VM

Here’s another use case you can try, which consists of snapshotting the VM. You can do this by adding the following Terraform code snippet in your configuration file, in the VM resource block:

```markdown
   snapshot {
    name        = "Snap1"
    description = "Snap this VM so we can restart from this state"
  }
```

Save the file, validate and apply those changes.

In [ ]:
terraform validate
terraform apply -input=false -auto-approve

Wait a little and check the details of the VM  once again, in the Backups section. You should have something similar to:

![Snap of VM ready](./Pictures/vmsnapped.png "Snap of VM ready")

### Use case 4: Stop this VM

Let's finish this lab with one last scenario. You might want to just turn off the VMs that are part of an infrastructure when you don’t need them to save cost or limit your carbon impact on the planet. If you paid attention to the configuration file, you’ll see that we inserted a power statement when originally creating the VM. While *poweron* is the only valid option when creating a new resource of type **hpegl\_vmaas\_instance**, other values are available for lifecycle management, such as *poweroff* and *suspend*.

Locate the following section in your configuration file:

```markdown
     }
     power = "poweron"
   }
```

And change it so that the power desired state is set to *poweroff*, as shown below:

```markdown
     }
     power = "poweroff"
   }
```

Save file, validate and apply changes a last time:

In [ ]:
terraform validate
terraform apply -input=false -auto-approve

Pretty soon afterwards, you can check out the HPE GreenLake console and see that the VM status was changed to stopped, as shown below:

![VM is now stopped](./Pictures/vmstopped.png "VM is now stopped")

## Debugging when things go wrong

In this lab, we’ve showed you how to make sure the Terraform configuration file is valid before applying changes using the **terraform validate** command. To see more details during an apply command, you can also enable Terraform debug by simply setting up the TF_LOG environment variable. I suggest setting it up to DEBUG but other supported values are: TRACE, INFO, WARN and ERROR.
 
```markdown
export TF_LOG=DEBUG
```

We can try with a **terraform validate** command:

In [ ]:
export TF_LOG=DEBUG
terraform validate

This terminates part 2 of this workshop. In this lab, we used Terraform to perform life cycle management actions on our VM instance. Let's move to [part 3](3-WKSHP-Terraform101-DecommisioningResources.ipynb) to see how we can decommision infrastructure with Terraform.
